In [39]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [41]:
import pandas as pd

## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("C:\\Users\\pbele\\Ironhack\\lab-natural-language-processing\\data\\kg_train.csv", encoding='latin-1')

import os
print(os.getcwd())

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

c:\Users\pbele\Ironhack\lab-natural-language-processing\your-code
(1000, 2)


### Let's divide the training and test set into two partitions

In [42]:
from sklearn.model_selection import train_test_split

# Assuming 'data' is the dataframe read in the previous steps.
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)


Training data shape: (800, 2)
Testing data shape: (200, 2)


## Data Preprocessing

In [43]:
import string
import nltk
nltk.download('stopwords')

print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pbele\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [44]:
import re
from bs4 import BeautifulSoup

def clean_html(text):
    # Remove JavaScript and CSS
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.S)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.S)
    # Remove remaining HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    return text

# Print column names to check
print(data.columns)

Index(['text', 'label'], dtype='object')


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [45]:
import re
from bs4 import BeautifulSoup

def clean_html(text):
    # Remove JavaScript and CSS
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.S)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.S)
    # Remove remaining HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    return text

# Apply the clean_html function to the 'text' column
data['cleaned_text'] = data['text'].apply(clean_html)


C:\Users\pbele\AppData\Local\Temp\ipykernel_25952\2184479602.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
C:\Users\pbele\AppData\Local\Temp\ipykernel_25952\2184479602.py:10: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


## Now let's work on removing stopwords
Remove the stopwords.

In [46]:
from nltk.corpus import stopwords

# Define stopwords set
stop_words = set(stopwords.words('english'))

# Define a function to remove stopwords
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

# Apply stopword removal to the 'cleaned_text' column and store the result in a new column 'processed_text'
data['processed_text'] = data['cleaned_text'].apply(remove_stopwords)


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [52]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

# Apply lemmatization
data['processed_text'] = data['processed_text'].apply(lemmatize_text)



## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)


In [62]:
from sklearn.feature_extraction.text import CountVectorizer

# Filter out empty rows to ensure non-empty text data
spam_messages = data[(data['label'] == 'spam') & (data['text'].str.strip() != '')]
ham_messages = data[(data['label'] == 'ham') & (data['text'].str.strip() != '')]

# Verify that there are non-empty messages in each category
if spam_messages.empty:
    print("No spam messages available for analysis.")
else:
    # Initialize vectorizer with a maximum of 10 features (top 10 words)
    vectorizer = CountVectorizer(max_features=10)
    spam_bow = vectorizer.fit_transform(spam_messages['text'])
    print("Top 10 words in spam messages:", vectorizer.get_feature_names_out())

if ham_messages.empty:
    print("No ham messages available for analysis.")
else:
    # Reinitialize the vectorizer for ham messages
    vectorizer = CountVectorizer(max_features=10)
    ham_bow = vectorizer.fit_transform(ham_messages['text'])
    print("Top 10 words in ham messages:", vectorizer.get_feature_names_out())


No spam messages available for analysis.
No ham messages available for analysis.


## Extra features

In [65]:
# Define indicators
money_symbols = r"euro|dollar|pound|€|\$"
suspicious_words = r"free|cheap|sex|money|account|bank|win|fund"

# Adding features for money symbols and suspicious words
data['money_mark'] = data['processed_text'].str.contains(money_symbols).astype(int)
data['suspicious_words'] = data['processed_text'].str.contains(suspicious_words).astype(int)
data['text_len'] = data['processed_text'].apply(len)

data.head()

,text,label,cleaned_text,processed_text,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...","DEAR SIR, STRICTLY PRIVATE BUSINESS PROPOSAL M...",1,0,1608
1,Will do.,0,Will do.,do.,0,0,3
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora--Cheryl has emailed dozens of memos about...,Nora--Cheryl emailed dozen memo Haiti weekend....,0,0,118
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir=2FMadam=2C I know that this proposal ...,Dear Sir=2FMadam=2C know proposal might surpri...,1,1,1524
4,fyi,0,fyi,fyi,0,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [67]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
count_vectorizer = CountVectorizer()

# Fit and transform the data
bow_data = count_vectorizer.fit_transform(data['processed_text'])

# Print the shape of the vectorized dataset
print("Shape of the Bag of Words dataset:", bow_data.shape)



Shape of the Bag of Words dataset: (1000, 25260)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the data
tfidf_data = tfidf_vectorizer.fit_transform(data['processed_text'])

# Print the shape of the vectorized dataset
print("Shape of the TF-IDF dataset:", tfidf_data.shape)


Shape of the TF-IDF dataset: (1000, 25260)


## And the Train a Classifier?

In [69]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Split into features and labels
X_train, X_test, y_train, y_test = train_test_split(tfidf_data, data['label'], test_size=0.2, random_state=42)

# Initialize and train classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Predict on test data
y_pred = classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the classifier:", accuracy)


Accuracy of the classifier: 0.95


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'data' contains the cleaned and preprocessed dataset with 'processed_text' and 'label' columns
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['processed_text'], data['label'], test_size=0.2, random_state=42)

# Trying both TF-IDF and Count Vectorizer to see which yields the best accuracy

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train the MultinomialNB classifier
tfidf_classifier = MultinomialNB()
tfidf_classifier.fit(X_train_tfidf, y_train)

# Predict and evaluate with TF-IDF
y_pred_tfidf = tfidf_classifier.predict(X_test_tfidf)
tfidf_accuracy = accuracy_score(y_test, y_pred_tfidf)
print("TF-IDF Vectorizer Accuracy:", tfidf_accuracy)
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf))

# Count Vectorizer
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# Train the classifier with Count Vectorizer
count_classifier = MultinomialNB()
count_classifier.fit(X_train_count, y_train)

# Predict and evaluate with Count Vectorizer
y_pred_count = count_classifier.predict(X_test_count)
count_accuracy = accuracy_score(y_test, y_pred_count)
print("Count Vectorizer Accuracy:", count_accuracy)
print("Classification Report (Count Vectorizer):\n", classification_report(y_test, y_pred_count))

# Determine the best feature representation
if tfidf_accuracy > count_accuracy:
    print("Best feature representation: TF-IDF Vectorizer with accuracy", tfidf_accuracy)
else:
    print("Best feature representation: Count Vectorizer with accuracy", count_accuracy)


TF-IDF Vectorizer Accuracy: 0.945
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           0       1.00      0.91      0.95       125
           1       0.87      1.00      0.93        75

    accuracy                           0.94       200
   macro avg       0.94      0.96      0.94       200
weighted avg       0.95      0.94      0.95       200

Count Vectorizer Accuracy: 0.94
Classification Report (Count Vectorizer):
               precision    recall  f1-score   support

           0       0.99      0.91      0.95       125
           1       0.87      0.99      0.93        75

    accuracy                           0.94       200
   macro avg       0.93      0.95      0.94       200
weighted avg       0.95      0.94      0.94       200

Best feature representation: TF-IDF Vectorizer with accuracy 0.945
